# **Analyzing CIA Factbook Data Using SQL**


##Introduction: <br>
In this project, I'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compilation of statistics about all of the countries on Earth. The Factbook contains demographic information like population, population growth, area, etc. I'll will be working with SQLite in this project to explore and analyze the database.


##Goals:
The goals of this project are the following: <br>
1. Work with data using SQL in Jupyter Notebook <br>
2. Generate summary statistics using SQL

First, I'll get started installing IPython.

In [ ]:
!conda install -yc conda-forge ipython-sql

Solving environment: done

## Package Plan ##

  environment location: /Users/User/opt/anaconda3

  added / updated specs:
    - ipython-sql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.5                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following packages will be UPDATED:

  conda                                4.8.4-py37hc8dfbb8_2 --> 4.8.5-py37hc8dfbb8_1



conda-4.8.5          | 3.0 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


I'll connect my Jupyter notebook to my database file.

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

I'll use a query to get information on the tables of this database.

In [ ]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type= 'table'

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


I have two tables here: sqlite_sequence and facts. For this project, I'll focus on the latter.

## **Overview of the Data**  
I'll start by getting a sense of what the data looks like.

In [ ]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

**name** - The name of the country. <br>
**area** - The total land and sea area of the country. <br>
**population** - The country's population. <br>
**population_growth**- The country's population growth as a percentage. <br>
**birth_rate** - The country's birth rate, or the number of births a year per 1,000 people. <br>
**death_rate** - The country's death rate, or the number of death a year per 1,000 people. <br>
**area**- The country's total area (both land and water). <br>
**area_land** - The country's land area in square kilometers. <br>
**area_water** - The country's waterarea in square kilometers. <br>



## **Summary Statistics**
I'll calculate some summary statistics on population and population growth.

In [ ]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
 FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen: <br>

There's a country with a population of 0. <br>

There's a country with a population of 7256490011 (or more than 7.2 billion people). <br>



## **Exploring Outliers**
I'll use subqueries to take a closer look at these countries without using the specific values.

In [ ]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                         FROM facts
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


It seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook page for Antarctica.

In [ ]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                         FROM facts
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None



I also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion I found earlier. <br>


## **Summary Statistics Revisited**
I'll now recalculate the summary statistics I calculated earlier, while excluding the row for the whole world.

In [ ]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
 FROM facts
WHERE name <> 'World'; 

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion!

What country has 1.4 billion people? What country has the highest growth rate?

In [ ]:
%%sql 
SELECT name, MAX(population) population
  FROM facts
    WHERE name NOT IN ("World");

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


In [ ]:
%%sql
SELECT name, MAX(population_growth) population_growth
  FROM facts;

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02


China has 1.4 billion people while South Sudan has the highest growth rate!

## **Exploring Average Population and Area**

I'll explore density next. Density depends on the population and the country's area. Look at the average values for these two columns. <br>

I'll discard the row for the whole planet.

In [ ]:
%%sql
SELECT AVG(population) AS avg_pop, AVG(area) AS avg_area
 FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


The average population is around 32 million and the average area is 555 thousand square kilometers.

## **Finding Densely Populated Countries**

I'll build on the query above to find countries that are densely populated and identify countries that have:<br>

Above average values for population.<br>

Below average values for area.

In [ ]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                         FROM facts
                    )
                 AND area < (SELECT AVG(area)
                         FROM facts
                    );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


Some of these countries are generally known to be densely populated!

## Exploring Population To Area Ratio

I'm going to explore which countries have the highest population to area ratio. This will tell me how densely populated this areas are!

In [ ]:
%%sql
SELECT name, area, population, ROUND(CAST(population AS FLOAT)/ area, 2) AS pop_area_ratio
 FROM facts
    ORDER BY pop_area_ratio DESC
    LIMIT 5;

 * sqlite:///factbook.db
Done.


name,area,population,pop_area_ratio
Macau,28,592731,21168.96
Monaco,2,30535,15267.5
Singapore,697,5674472,8141.28
Hong Kong,1108,7141106,6445.04
Gaza Strip,360,1869055,5191.82


Lastly, I'd like to know the average population density.

In [ ]:
%%sql
SELECT ROUND(AVG(CAST(population AS FLOAT)/ area_land), 2) AS avg_pop_area
 FROM facts
    WHERE name NOT IN ("Antarctica");

 * sqlite:///factbook.db
Done.


avg_pop_area
431.91


The average density is almost 432 square kilometers!

## Exploring Water to Land Ratios

Which countries have the highest ratios of water to land?

In [ ]:
%%sql
SELECT name, area_land, area_water, ROUND(CAST(area_water AS FLOAT)/ area_land, 2) AS water_land_ratio
 FROM facts
    ORDER BY water_land_ratio DESC
    LIMIT 5;

 * sqlite:///factbook.db
Done.


name,area_land,area_water,water_land_ratio
British Indian Ocean Territory,60,54340,905.67
Virgin Islands,346,1564,4.52
Puerto Rico,8870,4921,0.55
"Bahamas, The",10010,3870,0.39
Guinea-Bissau,28120,8005,0.28


British Indian Ocean Territory and The Virgin Islands have the highest water to land ratio!

Which countries have more water than land?

In [ ]:
%%sql
SELECT *
 FROM facts
    WHERE area_water > area_land;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


Not a surprise that the Islands have more water than land!